In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from scipy.stats import uniform
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
rcParams['figure.figsize'] = 12, 4
rcParams['lines.linewidth'] = 3
rcParams['xtick.labelsize'] = 'x-large'
rcParams['ytick.labelsize'] = 'x-large'


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from imblearn import under_sampling, over_sampling
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.base import BaseEstimator, RegressorMixin
from xgboost import XGBClassifier
import statsmodels.api as sm

## Load Data

In [2]:
df = pd.read_csv('../data/data_modeling.csv',sep=',')
df.head(2)

,age,default,balance,housing,loan,duration,campaign,previous,y,contact_group,...,marital_divorced,marital_married,marital_single,education_primary,education_secondary,education_tertiary,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,0,2143,1,0,4.350000,1,0,0,0,...,0,1,0,0,0,1,0,0,0,1
1,44,0,29,1,0,2.516667,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1


In [3]:
y = df['y'].values
X = df.drop(labels = ['y'], axis = 1)
print("Shape of X is {} and that of y is {}".format(X.shape, y.shape))

Shape of X is (45206, 25) and that of y is (45206,)


## Class Imbalance (SMOTE)

In [4]:
X_sm, y_sm = over_sampling.SMOTE(0.5).fit_resample(X, y)
print("Shape of X is {} and that of y is {}".format(X_sm.shape, y_sm.shape))

Shape of X is (59878, 25) and that of y is (59878,)


## Split Train-Test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size = 0.3, random_state = 42)

Add constraint for Logistic Regression using statsmodel

In [6]:
X_train_stat = sm.add_constant(X_train)
X_test_stat = sm.add_constant(X_test)

## Modeling

In [17]:
def eval_classification(model, xtrain, ytrain, xtest, ytest):
    ypred = model.predict(xtest)
    ypred = (ypred > 0.5).astype(int)
    y_train = model.predict(xtrain)
    y_train = (y_train > 0.5).astype(int)
    
    return accuracy_score(ytest, ypred), precision_score(ytest, ypred), recall_score(ytest, ypred), roc_auc_score(ytest, ypred), f1_score(ytest, ypred), f1_score(ytrain, y_train)
    
def show_feature_importance(model):
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

def show_best_hyperparameter(model, hyperparameters):
    for key, value in hyperparameters.items() :
        print('Best '+key+':', model.get_params()[key])

In [15]:
models = {
    "KNN" : KNeighborsClassifier(),
    "AdaBoost" :AdaBoostClassifier(random_state=42),
    "XGBoost" : XGBClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest" : RandomForestClassifier(random_state=42),
    "Logistic Regression" : sm.Logit(y_train,X_train_stat)
    }

In [33]:
scores = []
for model in models:
  print("Model "+model + " score:")
  classifier = models[model]
  if model!="Logistic Regression" :
    classifier.fit(X_train, y_train)
    score = eval_classification(classifier,X_train,y_train,X_test,y_test)
  else:
    result = classifier.fit_regularized(random_state = 42)
    score = eval_classification(result, X_train_stat, y_train, X_test_stat, y_test)

  scores.append([model,score[0],score[1], score[2], score[3], score[4], score[5]])

  print("Accuracy (Test Set): %.2f" % score[0])
  print("Precision (Test Set): %.2f" % score[1])
  print("Recall (Test Set): %.2f" % score[2])
  print("AUC (Test Set) : %.2f" % score[3])
  print("F1-Score (Test Set): %.2f" % score[4])
  print("F1-Score (Train Set): %.2f" % score[5])
  print()

Model KNN score:
Accuracy (Test Set): 0.85
Precision (Test Set): 0.74
Recall (Test Set): 0.85
AUC (Test Set) : 0.85
F1-Score (Test Set): 0.79
F1-Score (Train Set): 0.86

Model AdaBoost score:
Accuracy (Test Set): 0.90
Precision (Test Set): 0.88
Recall (Test Set): 0.82
AUC (Test Set) : 0.88
F1-Score (Test Set): 0.85
F1-Score (Train Set): 0.85

Model XGBoost score:
[23:25:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy (Test Set): 0.91
Precision (Test Set): 0.90
Recall (Test Set): 0.84
AUC (Test Set) : 0.89
F1-Score (Test Set): 0.87
F1-Score (Train Set): 0.92

Model Decision Tree score:
Accuracy (Test Set): 0.88
Precision (Test Set): 0.81
Recall (Test Set): 0.82
AUC (Test Set) : 0.86
F1-Score (Test Set): 0.82
F1-Score (Train Set

In [38]:
eval = pd.DataFrame(scores)
eval = eval.rename({0:'Model', 1: 'Accuracy', 2: 'Precision', 3: 'Recall', 4: 'AUC', 5: 'F1(test)', 6: 'F1(train)'}, axis = 1)
eval.sort_values(['F1(test)','F1(train)'], ascending=False).reset_index().drop('index', axis = 1)

,Model,Accuracy,Precision,Recall,AUC,F1(test),F1(train)
0,XGBoost,0.914774,0.901337,0.835091,0.894779,0.866951,0.920701
1,Random Forest,0.913605,0.893537,0.840281,0.895205,0.866091,0.999964
2,Logistic Regression,0.909374,0.924068,0.792567,0.880063,0.853280,0.863640
3,AdaBoost,0.902694,0.881249,0.817512,0.881319,0.848185,0.853726
4,Decision Tree,0.876364,0.809367,0.821698,0.862646,0.815486,1.000000
5,KNN,0.851982,0.742251,0.849992,0.851482,0.792476,0.863122
